# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [60]:
# Dependencies and Setup

import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import cartopy
import geoviews
import json

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,23.03,64,75,5.14,CK,1697416131
1,1,heihe,50.2441,127.4902,2.32,42,8,8.61,CN,1697416131
2,2,bethel,41.3712,-73.4140,11.47,79,100,2.06,US,1697416132
3,3,mukalla,14.5425,49.1242,26.05,80,77,1.19,YE,1697416132
4,4,albany,42.6001,-73.9662,9.76,81,99,2.24,US,1697415857


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [111]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[ (city_data_df["Max Temp"] >= 22) &  (city_data_df["Humidity"] <= 60) ]
clean_ideal_city_df = ideal_df[ ideal_df["Cloudiness"] == 0 ] 

# Drop any rows with null values
clean_ideal_data_df = clean_ideal_city_df.dropna()

# Display sample data
clean_ideal_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,ponta pora,-22.5361,-55.7256,26.95,57,0,3.60,BR,1697416134
46,46,rawah,34.4816,41.9093,22.24,46,0,4.35,IQ,1697416144
152,152,laguna,38.4210,-121.4238,30.59,29,0,1.54,US,1697416176
160,160,malibu,34.0050,-118.8101,33.71,46,0,4.22,US,1697416180
189,189,san patricio,28.0170,-97.5169,24.92,32,0,5.14,US,1697416190
225,225,aswan,24.0934,32.9070,24.02,32,0,4.74,EG,1697416201
243,243,naze,28.3667,129.4833,23.51,50,0,5.28,JP,1697416208
296,296,al artawiyah,26.5053,45.3444,24.47,17,0,1.57,SA,1697416225
339,339,mount isa,-20.7333,139.5000,29.87,16,0,8.23,AU,1697416230
365,365,san sebastian de la gomera,28.0916,-17.1133,28.20,29,0,0.00,ES,1697416026


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,ponta pora,BR,-22.5361,-55.7256,57,
46,rawah,IQ,34.4816,41.9093,46,
152,laguna,US,38.4210,-121.4238,29,
160,malibu,US,34.0050,-118.8101,46,
189,san patricio,US,28.0170,-97.5169,32,
225,aswan,EG,24.0934,32.9070,32,
243,naze,JP,28.3667,129.4833,50,
296,al artawiyah,SA,26.5053,45.3444,17,
339,mount isa,AU,-20.7333,139.5000,16,
365,san sebastian de la gomera,ES,28.0916,-17.1133,29,


In [85]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Reset Index
newindex = np.arange(0,len(hotel_df))
hotel_df = hotel_df.set_index(newindex)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ponta pora,BR,-22.5361,-55.7256,57,
1,rawah,IQ,34.4816,41.9093,46,
2,laguna,US,38.4210,-121.4238,29,
3,malibu,US,34.0050,-118.8101,46,
4,san patricio,US,28.0170,-97.5169,32,
5,aswan,EG,24.0934,32.9070,32,
6,naze,JP,28.3667,129.4833,50,
7,al artawiyah,SA,26.5053,45.3444,17,
8,mount isa,AU,-20.7333,139.5000,16,
9,san sebastian de la gomera,ES,28.0916,-17.1133,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [90]:
# Set parameters to search for a hotel

radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row.loc['Lat']
    city_lng = row.loc['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"circle:{city_lng},{city_lat},{radius}"
    bias = f"proximity:{city_lng},{city_lat}"
    params["filter"] = filter
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ponta pora - nearest hotel: Pousada do Bosque
rawah - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
malibu - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
naze - nearest hotel: ビジネス旅館畠山
al artawiyah - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
del rio - nearest hotel: La Quinta Inn by Wyndham Del Rio
akjoujt - nearest hotel: فندق فاطمة
ruteng - nearest hotel: No hotel found
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
umluj - nearest hotel: No hotel found
blythe - nearest hotel: Quality Inn
siwa oasis - nearest hotel: فندق الكيلانى
eilat - nearest hotel: Melony Club
manacapuru - nearest hotel: Novo Hotel
darwin - nearest hotel: Mantra Pantanas Darwin


,City,Country,Lat,Lng,Humidity,Hotel Name
0,ponta pora,BR,-22.5361,-55.7256,57,Pousada do Bosque
1,rawah,IQ,34.4816,41.9093,46,No hotel found
2,laguna,US,38.4210,-121.4238,29,Holiday Inn Express & Suites
3,malibu,US,34.0050,-118.8101,46,No hotel found
4,san patricio,US,28.0170,-97.5169,32,No hotel found
5,aswan,EG,24.0934,32.9070,32,Yaseen hotel
6,naze,JP,28.3667,129.4833,50,ビジネス旅館畠山
7,al artawiyah,SA,26.5053,45.3444,17,No hotel found
8,mount isa,AU,-20.7333,139.5000,16,Ibis Styles
9,san sebastian de la gomera,ES,28.0916,-17.1133,29,Hotel Villa Gomera


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [113]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ['City','Humidity','Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)